In [ ]:
import networkx as nx
import gzip
import ujson as json
import itertools
# import treelib
import numpy as np
#import matplotlib.pyplot as plt
from torch_geometric.utils.convert import from_networkx
#import torch_geometric.utils.convert
from torch_geometric.loader import DataLoader
import pandas as pd
import torch

In [2]:
print(torch.__version__)


1.11.0


In [3]:
df1=pd.read_csv("/Users/abdulmk/Projects/Capstone/Datasets/finalirma.csv")
df1.columns

Index(['id', 'image_id', 'text_info', 'text_info_conf', 'image_info',
       'image_info_conf', 'text_human', 'text_human_conf', 'image_human',
       'image_human_conf', 'image_damage', 'image_damage_conf', 'tweet_text',
       'image_url', 'image_path', 'coordinates', 'created_at', 'hashtags',
       'media', 'urls', 'favorite_count', 'in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_user_id', 'lang', 'place',
       'possibly_sensitive', 'quote_id', 'retweet_count', 'retweet_id',
       'retweet_screen_name', 'source', 'text', 'tweet_url', 'user_created_at',
       'user_id', 'user_default_profile_image', 'user_description',
       'user_favourites_count', 'user_followers_count', 'user_friends_count',
       'user_listed_count', 'user_location', 'user_name', 'user_screen_name',
       'user_statuses_count', 'user_time_zone', 'user_urls', 'user_verified'],
      dtype='object')

In [4]:
df1

,id,image_id,text_info,text_info_conf,image_info,image_info_conf,text_human,text_human_conf,image_human,image_human_conf,...,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name,user_statuses_count,user_time_zone,user_urls,user_verified
0,905625009430949888,905625009430949888_0,informative,0.6721,informative,1.0000,infrastructure_and_utility_damage,0.6721,infrastructure_and_utility_damage,1.0000,...,316,756,4,Jamaica,Cecile,slimjamaican,40303,NaN,NaN,False
1,905625059326353409,905625059326353409_0,informative,0.6545,informative,1.0000,other_relevant_information,0.6545,other_relevant_information,1.0000,...,53361,1999,408,"Washington, DC",Steve Rudin 7News,SteveRudin7News,30933,NaN,http://wjla.com/station/people/steve-rudin,True
2,905625064451833856,905625064451833856_0,informative,0.6629,informative,0.6392,infrastructure_and_utility_damage,0.6629,other_relevant_information,0.6392,...,34,179,0,Toronto,Christine 🌸,kiki_chiquita,490,NaN,NaN,False
3,905625073628975106,905625073628975106_0,informative,1.0000,informative,1.0000,other_relevant_information,1.0000,other_relevant_information,1.0000,...,597,2962,9,"Florida, USA",Laura Dill,Perspectvz,62816,NaN,NaN,False
4,905625088300654592,905625088300654592_0,informative,1.0000,informative,1.0000,other_relevant_information,1.0000,other_relevant_information,1.0000,...,2220,4298,68,Chile,Edhagadol,edhagadol,59091,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3515,910270036861886465,910270036861886465_0,informative,1.0000,informative,1.0000,other_relevant_information,1.0000,infrastructure_and_utility_damage,1.0000,...,406,243,8,NaN,On Weather,OnWeather1,71804,NaN,NaN,False
3516,910270462822895617,910270462822895617_0,informative,1.0000,not_informative,0.3779,other_relevant_information,1.0000,not_humanitarian,0.3779,...,591,61,23,"Kingsland, GA",Camden County EMA,CamdenCountyEMA,4635,NaN,https://www.camdencountyga.gov/81/Emergency-Ma...,False
3517,910270554812317697,910270554812317697_0,informative,0.6376,informative,0.6445,other_relevant_information,0.6376,other_relevant_information,0.6445,...,1090,709,29,(223/717),Nikhil Menon,multimenon,41946,NaN,https://nikhilmenon.in,False
3518,910270595262291968,910270595262291968_0,not_informative,0.6667,not_informative,1.0000,not_humanitarian,0.6667,not_humanitarian,1.0000,...,1533,3859,183,"Wewoka, OK",Jake Davis,WewokaChronicle,208925,NaN,http://writers.work/jakedavis,False


In [5]:
p=df1.loc[df1['id']==910168389846536193,['text','created_at']]
print(p.values[0])

['Hurricane Irma uprooted a tree in a cemetery, its roots unearthed a casket 😱😞 #grave #irma https://t.co/cOJ1VYPsYR'
 'Tue Sep 19 15:47:08 +0000 2017']


In [16]:
s='Tue Sep 19 15:47:08 +0000 2017'
ts_list = pd.to_datetime([s], infer_datetime_format=True)
#print(ts_list.values)
import dateutil.parser as parser
parser.parse(s)

def time_parser(ts):
    dobject=parser.parse(ts)
    return 

datetime.datetime(2017, 9, 19, 15, 47, 8, tzinfo=tzutc())

In [7]:
import os
cwd="/Users/abdulmk/Projects/Capstone/convos"
data_arr=[]
for filename in os.listdir(cwd):
    fname=os.path.join(cwd, filename)
    #print(fname)
    data = pd.read_json(fname, lines=True)
    rootid=int(filename[:-6])
    r=df1.loc[df1['id']==rootid,['text','created_at','text_info']]
    #print(r)
    t,c,label=r.values[0]
    d={}
    d[rootid]={'time':c,'text':t}

    G=nx.DiGraph()
    G.add_node(rootid)
    for i in data['data'][0]:
        G.add_node(int(i['id']))
        d[int(i['id'])]={'time':i['created_at'],'text':i['text']}
        G.add_edge(int(i['id']),int(i['referenced_tweets'][0]['id']))
    #print(G.edges)
    #print(filename)
    
    H = G.to_undirected()
    Gcc = sorted(nx.connected_components(H), key=len, reverse=True)
    G0 = G.subgraph(Gcc[0])
    G1=G0.to_directed()
    #nx.draw(G1,with_labels=True)
    nx.set_node_attributes(G1, d)
    #print(nx.get_node_attributes(G1,'text'))
    #print(d)
    try:
        g_data=from_networkx(G1)
        g_data.y=1 if label=='informative' else 0
        #print(g_data)
        data_arr.append(g_data)
    except:
        continue
    
        
print(len(data_arr))
    


320


In [8]:
dataset=data_arr
data=dataset[0]

In [9]:

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Data(edge_index=[2, 1], time=[2], text=[2], num_nodes=2, y=1)
Number of nodes: 2
Number of edges: 1
Average node degree: 0.50
Has isolated nodes: False
Has self-loops: False
Is undirected: False


In [10]:
import torch
from random import shuffle
torch.manual_seed(12345)
shuffle(dataset)

train_dataset = dataset[:260]
test_dataset = dataset[260:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 260
Number of test graphs: 60


In [11]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 239], time=[64], text=[64], num_nodes=303, y=[64], batch=[303], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 158], time=[64], text=[64], num_nodes=222, y=[64], batch=[222], ptr=[65])

Step 3:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 236], time=[64], text=[64], num_nodes=300, y=[64], batch=[300], ptr=[65])

Step 4:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 245], time=[64], text=[64], num_nodes=309, y=[64], batch=[309], ptr=[65])

Step 5:
Number of graphs in the current batch: 4
DataBatch(edge_index=[2, 8], time=[4], text=[4], num_nodes=12, y=[4], batch=[12], ptr=[5])

